<a href="https://colab.research.google.com/github/OberoiHimanshu/CockpitNextGen/blob/Deep-Learning/05_April_transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0-rc2'

In [3]:
! mkdir -p /root/.kaggle
! mv kaggle.json /root/.kaggle
import kaggle

In [4]:
! kaggle competitions download -c dogs-vs-cats

 98% 266M/271M [00:04<00:00, 99.0MB/s]
100% 271M/271M [00:04<00:00, 65.2MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 71.0MB/s]
 96% 523M/543M [00:06<00:00, 81.9MB/s]
100% 543M/543M [00:06<00:00, 92.9MB/s]


In [0]:
! unzip -q train.zip

In [0]:
# ! ls train

In [0]:
import os 
os.mkdir("data")
os.mkdir("data/cat")
os.mkdir("data/dog")

In [0]:
SRC_FOLDER = "train"
CAT_FODLER = "data/cat"
DOG_FOLDER = "data/dog"

In [8]:
from  shutil import copy
from tqdm import tqdm_notebook
for image in tqdm_notebook(os.listdir(SRC_FOLDER)):
  if image.lower().startswith("cat"):
    copy(os.path.join(SRC_FOLDER, image), os.path.join(CAT_FODLER))
  else:
    copy(os.path.join(SRC_FOLDER, image), os.path.join(DOG_FOLDER))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# ! ls data/cat

In [0]:
from tensorflow.keras.applications import VGG16

In [10]:
model = VGG16(include_top=False, input_shape=(224, 224, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [11]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
for layer in model.layers:
  layer.trainable = False
for layer in model.layers:
  print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [13]:
model.output_shape[1:]

(7, 7, 512)

In [0]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy

In [15]:
from tensorflow.keras import __version__
__version__

'2.3.0-tf'

In [0]:
# top_model = Sequential()
# top_model.add(Flatten(input_shape=(model.output_shape[1:])))
# top_model.add(Dense(256, activation=relu))
# top_model.add(Dropout(0.4))
# top_model.add(Dense(2, activation=softmax))

In [16]:
model.layers[-1].output

<tf.Tensor 'block5_pool/Identity:0' shape=(None, 7, 7, 512) dtype=float32>

In [0]:
type(top_model), type(model)

(tensorflow.python.keras.engine.sequential.Sequential,
 tensorflow.python.keras.engine.training.Model)

In [0]:
x = model.layers[-1].output
x = Flatten()(x)
x = Dense(256, relu)(x)
x = Dropout(0.4)(x)
x = Dense(2, softmax)(x)

In [0]:
from tensorflow.keras.models import Model

In [0]:
final_model = Model(inputs=model.input, outputs=x)

In [20]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
for layer in final_model.layers:
  print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten True
dense True
dropout True
dense_1 True


In [0]:
final_model.compile("rmsprop", categorical_crossentropy, ["acc"])

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(rescale=1/255.)
train_gen = idg.flow_from_directory("data", target_size=(224, 224), batch_size=128)

Found 25000 images belonging to 2 classes.


In [24]:
final_model.fit_generator(train_gen, steps_per_epoch=25000//128, epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
195/195 [==============================] - 221s 1s/step - loss: 1.0191 - acc: 0.7945
Epoch 2/10
195/195 [==============================] - 221s 1s/step - loss: 0.2748 - acc: 0.8876
Epoch 3/10
195/195 [==============================] - 220s 1s/step - loss: 0.2225 - acc: 0.9093
Epoch 4/10
195/195 [==============================] - 220s 1s/step - loss: 0.1844 - acc: 0.9249
Epoch 5/10
195/195 [==============================] - 220s 1s/step - loss: 0.1746 - acc: 0.9290
Epoch 6/10
195/195 [==============================] - 220s 1s/step - loss: 0.1603 - acc: 0.9353
Epoch 7/10
195/195 [==============================] - 219s 1s/step - loss: 0.1446 - acc: 0.9417
Epoch 8/10
195/195 [==============================] - 220s 1s/step - loss: 0.1369 - acc: 0.9452
Epoch 9/10
195/195 [==============================] - 220s 1s/step - loss: 0.1251 - acc: 0.9496
Epoch 10/10
195/195 [==============================] - 220s 

In [25]:
final_model.save("First_vgg16_Model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: First_vgg16_Model/assets
